<a href="https://colab.research.google.com/github/shuchimishra/Tensorflow_projects/blob/main/Tensorflow_Code/Intro_to_ANN/exam/Category2_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Question
# #
# # Create a classifier for the CIFAR10 dataset
# # Note that the test will expect it to classify 10 classes and that the input shape should be
# # the native CIFAR size which is 32x32 pixels with 3 bytes color depth

# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import *
# def solution_model():
#     # cifar = tf.keras.datasets.cifar10
#     (x_train,y_train), (x_test,y_test) =  tf.keras.datasets.cifar10.load_data()
#     #2.모델구성
#     model = Sequential()
#     model.add(Conv2D(16,(3,3),input_shape = (32,32,3),activation='swish'))
#     model.add(Conv2D(24,(3,3),activation='swish'))
#     model.add(Conv2D(32,(3,3),activation='swish'))
#     model.add(Flatten())
#     model.add(Dense(10,activation='softmax'))

#     #3.컴파일 훈련
#     model.compile(loss='SparseCategorical_Crossentropy',optimizer='adam',metrics='accuracy')
#     model.fit(x_train,y_train,epochs=100,verbose=1,validation_split=0.1)

#     loss = model.evaluate(x_test,y_test)
#     print('loss:',loss[0])
#     print('acc:',loss[1])

#     return model



# # Note that you'll need to save your model as a .h5 like this
# # This .h5 will be uploaded to the testing infrastructure
# # and a score will be returned to you
# if __name__ == '__main__':
#     model = solution_model()
#     model.save("mymodel.h5")

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense

def solution_model():
    # 데이터 불러오기
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # 모델 구성
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(Conv2D(24, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    # 컴파일, 훈련
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, verbose=1, validation_split=0.1)

    # 모델 평가
    loss, accuracy = model.evaluate(x_test, y_test)
    print('loss:', loss)
    print('accuracy:', accuracy)

    return model

# 모델 생성 및 저장
if __name__ == '__main__':
    model = solution_model()
    # model.save("mymodel.h5")
'''
loss: 3.0625479221343994
accuracy: 0.49950000643730164
'''

In [64]:
# Question
#
# Create a classifier for the CIFAR10 dataset
# Note that the test will expect it to classify 10 classes and that the input shape should be
# the native CIFAR size which is 32x32 pixels with 3 bytes color depth

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

class myCallBack(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs['val_accuracy'] > 0.99):
      print("Cancelling training since accuracy reached 99%")
      self.model.stop_training=True

def solution_model():

  (train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.cifar10.load_data()

  #number of classes
  n_classes = len(set(train_labels[:,0]))

  #normalize image
  train_images = train_images/255.0
  test_images = test_images/255.0

  #callbacks
  callback = myCallBack()
  MCP = keras.callbacks.ModelCheckpoint(filepath='bestmodel.h5',monitor='val_accuracy',
                                        mode='auto',save_best_only=True,save_weights_only=False,verbose=1)
  RLP = keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",factor=0.1,patience=3,
                                          verbose=1,mode="auto",min_lr=0.000000001)
  ES = keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=1e-4,patience=5,verbose=1)

  model = keras.models.Sequential([
      keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(32, 32, 3)), #filter=64
      keras.layers.MaxPooling2D((2,2)),
      keras.layers.Conv2D(128, (3,3), activation='relu'),
      keras.layers.MaxPooling2D((2,2)),
      keras.layers.Conv2D(128, (3,3), activation='relu'),
      keras.layers.MaxPooling2D((2,2)),
      # keras.layers.Dropout(0.5),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.2),
      # keras.layers.Dense(1024, activation='relu'), #512
      # keras.layers.Dense(512, activation='relu'), #512
      keras.layers.Dense(128, activation='relu'), #512
      keras.layers.Dense(n_classes, activation='softmax')])

  lr = 0.0001

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')

  model.summary()

  tf.keras.backend.clear_session()

  history = model.fit(
        train_images, train_labels,
        # batch_size=512,
        epochs=50,
        verbose=1,
        validation_data=(test_images, test_labels),
        callbacks=[callback,MCP,RLP,ES] #RLP,ES
    )

  # model.summary()
  #Evaluate against test images
  model.evaluate(test_images, test_labels)

  return model



# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         147584    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0

KeyboardInterrupt: 

In [33]:
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Conv2D, Flatten, Dense
# from tensorflow import keras
# import warnings
# warnings.filterwarnings('ignore')


In [15]:
# class myCallBack(keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if (logs['val_accuracy'] > 0.99):
#       print("Cancelling training since accuracy reached 99%")
#       self.model.stop_training=True

In [2]:
# (train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 17s 0us/step


In [7]:
# #number of classes
# n_classes = len(set(train_labels[:,0]))

In [9]:
# #normalize image
# train_images = train_images/255.0
# test_images = test_images/255.0

In [42]:
# print(train_images.shape)

(50000, 32, 32, 3)


In [59]:
# #callbacks
# callback = myCallBack()
# MCP = keras.callbacks.ModelCheckpoint(filepath='bestmodel.h5',monitor='val_accuracy',
#                                       mode='auto',save_best_only=True,save_weights_only=False,verbose=1)
# RLP = keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",factor=0.1,patience=3,
#                                         verbose=1,mode="auto",min_lr=0.000000001)
# ES = keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=1e-4,patience=5,verbose=1)

In [60]:
# model = keras.models.Sequential([
#     keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(32, 32, 3)), #filter=64
#     keras.layers.MaxPooling2D((2,2)),
#     keras.layers.Conv2D(128, (3,3), activation='relu'),
#     keras.layers.MaxPooling2D((2,2)),
#     keras.layers.Conv2D(128, (3,3), activation='relu'),
#     keras.layers.MaxPooling2D((2,2)),
#     # keras.layers.Dropout(0.5),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.2),
#     # keras.layers.Dense(1024, activation='relu'), #512
#     # keras.layers.Dense(512, activation='relu'), #512
#     keras.layers.Dense(128, activation='relu'), #512
#     keras.layers.Dense(n_classes, activation='softmax')])

# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         147584    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0

In [61]:
# lr = 0.0001

# optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# model.compile(optimizer=optimizer,
#                 loss='sparse_categorical_crossentropy',
#                 metrics='accuracy')

In [62]:
# tf.keras.backend.clear_session()

# history = model.fit(
#       train_images, train_labels,
#       # batch_size=512,
#       epochs=50,
#       verbose=1,
#       validation_data=(test_images, test_labels),
#       callbacks=[callback,MCP,RLP,ES] #RLP,ES
#   )

# # model.summary()
# #Evaluate against test images
# model.evaluate(test_images, test_labels)

Epoch 1/50
1563/1563 [==============================] - ETA: 0s - loss: 1.7712 - accuracy: 0.3493
Epoch 1: val_accuracy improved from -inf to 0.46660, saving model to bestmodel.h5
1563/1563 [==============================] - 16s 9ms/step - loss: 1.7712 - accuracy: 0.3493 - val_loss: 1.4851 - val_accuracy: 0.4666 - lr: 1.0000e-04
Epoch 2/50
1553/1563 [============================>.] - ETA: 0s - loss: 1.4349 - accuracy: 0.4819
Epoch 2: val_accuracy improved from 0.46660 to 0.52320, saving model to bestmodel.h5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4347 - accuracy: 0.4820 - val_loss: 1.3172 - val_accuracy: 0.5232 - lr: 1.0000e-04
Epoch 3/50
1553/1563 [============================>.] - ETA: 0s - loss: 1.3134 - accuracy: 0.5314
Epoch 3: val_accuracy improved from 0.52320 to 0.56380, saving model to bestmodel.h5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.3129 - accuracy: 0.5316 - val_loss: 1.2223 - val_accuracy: 0.5638 - lr: 1.0000e-04
E

[0.6964036226272583, 0.7718999981880188]

In [63]:
# model.save("mymodel.h5")